# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [2]:
# Import libraries
import numpy as np
import pandas as pd

In [3]:
# Importing the dataset
pokemon = pd.read_csv("Pokemon.csv")
pokemon.head(3)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [20]:
# your code here

# Create column total_hip with a hypothesis on how Total is computed:
col_num = ["HP","Attack","Defense","Sp. Atk","Sp. Def","Speed"]
pokemon["total_hip"] = pokemon[col_num].sum(axis=1)

#Testing
pokemon[(pokemon["total_hip"] == pokemon["Total"]) == False].value_counts()

#Conclusion:
# Total is the sum of "HP","Attack","Defense","Sp. Atk","Sp. Def" and "Speed"


Series([], dtype: int64)

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [36]:
# your code here


pok_dummies = pd.get_dummies(pokemon, columns= ['Type 1','Type 2'], prefix="",prefix_sep="")
display(pok_dummies)
pok_dummies.columns

,#,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,1,Bulbasaur,318,45,49,49,65,65,45,1,...,0,0,0,0,0,1,0,0,0,0
1,2,Ivysaur,405,60,62,63,80,80,60,1,...,0,0,0,0,0,1,0,0,0,0
2,3,Venusaur,525,80,82,83,100,100,80,1,...,0,0,0,0,0,1,0,0,0,0
3,3,VenusaurMega Venusaur,625,80,100,123,122,120,80,1,...,0,0,0,0,0,1,0,0,0,0
4,4,Charmander,309,39,52,43,60,50,65,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,600,50,100,150,100,150,50,6,...,0,0,0,0,0,0,0,0,0,0
796,719,DiancieMega Diancie,700,50,160,110,160,110,110,6,...,0,0,0,0,0,0,0,0,0,0
797,720,HoopaHoopa Confined,600,80,110,60,150,130,70,6,...,1,0,0,0,0,0,0,0,0,0
798,720,HoopaHoopa Unbound,680,80,160,60,170,130,80,6,...,0,0,0,0,0,0,0,0,0,0


Index(['#', 'Name', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def',
       'Speed', 'Generation', 'Legendary', 'total_hip', 'Bug', 'Dark',
       'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost',
       'Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock',
       'Steel', 'Water', 'Bug', 'Dark', 'Dragon', 'Electric', 'Fairy',
       'Fighting', 'Fire', 'Flying', 'Ghost', 'Grass', 'Ground', 'Ice',
       'Normal', 'Poison', 'Psychic', 'Rock', 'Steel', 'Water'],
      dtype='object')

In [45]:
display(pok_dummies.columns.value_counts())

pok_dummies.columns.duplicated()

Electric      2
Fairy         2
Steel         2
Rock          2
Psychic       2
Poison        2
Normal        2
Ice           2
Ground        2
Grass         2
Ghost         2
Flying        2
Fire          2
Fighting      2
Water         2
Dragon        2
Dark          2
Bug           2
Name          1
total_hip     1
Legendary     1
Generation    1
Speed         1
Sp. Def       1
Sp. Atk       1
Defense       1
Attack        1
HP            1
Total         1
#             1
dtype: int64

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [50]:
pok_dummies[["Electric"]].sum()

Electric    44
Electric     6
dtype: int64

In [51]:
pok_dum_final = pok_dummies.groupby(axis=1, level=0).sum()
pok_dum_final[["Electric"]].sum()

Electric    50
dtype: int64

In [54]:
pok_types = ['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric','Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice', 'Dragon', 'Dark', 'Steel', 'Flying']
pok_dum_final[pok_types]

,Grass,Fire,Water,Bug,Normal,Poison,Electric,Ground,Fairy,Fighting,Psychic,Rock,Ghost,Ice,Dragon,Dark,Steel,Flying
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
796,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
797,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
798,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [62]:
# your code here
pok_dum_final.sort_values(by= 'Total', ascending=False)

lables = ["Very Weak", "Weak", "Moderate", "Strong", "Very Strong"]

bins = pd.cut(pok_dum_final['Total'],5,labels=lables)
bins.value_counts()

Moderate       313
Weak           241
Very Weak      121
Strong          93
Very Strong     32
Name: Total, dtype: int64

In [67]:
pok_dum_final.groupby([bins])[pok_types].agg(sum)


,Grass,Fire,Water,Bug,Normal,Poison,Electric,Ground,Fairy,Fighting,Psychic,Rock,Ghost,Ice,Dragon,Dark,Steel,Flying
Total,,,,,,,,,,,,,,,,,,
Very Weak,12,3,13,20,26,12,7,12,10,5,12,4,4,3,6,5,3,13
Weak,35,24,46,24,29,23,12,16,9,12,18,22,14,9,8,13,8,26
Moderate,41,22,54,23,38,25,24,29,13,24,31,24,23,19,9,23,26,38
Strong,7,12,10,5,6,2,6,7,6,11,21,6,3,5,13,7,10,18
Very Strong,0,3,3,0,3,0,1,3,2,1,8,2,2,2,14,3,2,6


In [ ]:
# the #1 pokemon type that most likely have the highest `Total` are the type DRAGON!

# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [4]:
# your code here
pokemon['combo_type'] = np.where(pokemon['Type 2'].isna(),pokemon['Type 1'],(pokemon['Type 1'] + "-" + pokemon['Type 2']))

pok_dum_t1t2 = pd.get_dummies(pokemon, columns= ['combo_type'], prefix="",prefix_sep="")
display(pok_dum_t1t2)
col_lis_all = list(pok_dum_t1t2.columns)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,...,Water-Fighting,Water-Flying,Water-Ghost,Water-Grass,Water-Ground,Water-Ice,Water-Poison,Water-Psychic,Water-Rock,Water-Steel
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,...,0,0,0,0,0,0,0,0,0,0
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,...,0,0,0,0,0,0,0,0,0,0
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,...,0,0,0,0,0,0,0,0,0,0
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,...,0,0,0,0,0,0,0,0,0,0
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,...,0,0,0,0,0,0,0,0,0,0
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,...,0,0,0,0,0,0,0,0,0,0
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,...,0,0,0,0,0,0,0,0,0,0
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,...,0,0,0,0,0,0,0,0,0,0


In [5]:
rev_list= ['#', 'Name', 'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary', 'total_hip','Total'] 

lable_t1t2 = [x for x in col_lis_all if x not in rev_list]
lable_t1t2


['Bug',
 'Bug-Electric',
 'Bug-Fighting',
 'Bug-Fire',
 'Bug-Flying',
 'Bug-Ghost',
 'Bug-Grass',
 'Bug-Ground',
 'Bug-Poison',
 'Bug-Rock',
 'Bug-Steel',
 'Bug-Water',
 'Dark',
 'Dark-Dragon',
 'Dark-Fighting',
 'Dark-Fire',
 'Dark-Flying',
 'Dark-Ghost',
 'Dark-Ice',
 'Dark-Psychic',
 'Dark-Steel',
 'Dragon',
 'Dragon-Electric',
 'Dragon-Fairy',
 'Dragon-Fire',
 'Dragon-Flying',
 'Dragon-Ground',
 'Dragon-Ice',
 'Dragon-Psychic',
 'Electric',
 'Electric-Dragon',
 'Electric-Fairy',
 'Electric-Fire',
 'Electric-Flying',
 'Electric-Ghost',
 'Electric-Grass',
 'Electric-Ice',
 'Electric-Normal',
 'Electric-Steel',
 'Electric-Water',
 'Fairy',
 'Fairy-Flying',
 'Fighting',
 'Fighting-Dark',
 'Fighting-Flying',
 'Fighting-Psychic',
 'Fighting-Steel',
 'Fire',
 'Fire-Dragon',
 'Fire-Fighting',
 'Fire-Flying',
 'Fire-Ground',
 'Fire-Normal',
 'Fire-Psychic',
 'Fire-Rock',
 'Fire-Steel',
 'Fire-Water',
 'Flying',
 'Flying-Dragon',
 'Ghost',
 'Ghost-Dark',
 'Ghost-Dragon',
 'Ghost-Fire',
 'Gho

In [6]:
lables = ["Very Weak", "Weak", "Moderate", "Strong", "Very Strong"]

bins = pd.cut(pok_dum_t1t2['Total'],5,labels=lables)
print(bins.value_counts())

table = pok_dum_t1t2.groupby([bins])[lable_t1t2].agg(sum).reset_index()

Moderate       313
Weak           241
Very Weak      121
Strong          93
Very Strong     32
Name: Total, dtype: int64


In [36]:
table =table.set_index("Total")

In [44]:
table.loc["Very Strong"][table.loc["Very Strong"] >2]

Dragon-Flying    3
Normal           3
Name: Very Strong, dtype: uint8

In [ ]:
.corr()